In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import architectures

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(architectures)    
net = architectures.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, encoder_layers = 2,
                                            decoder_hidden_units = 100, decoder_layers = 2)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3, 
        save_path = "checkpoints/seq2seq_rnn.pt")

Net parameters: 408,795



Epoch:    1, Loss: 2.6771
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']

forward
['en  ee   e', 'ee ee   e ', '   e   e  ', '  n   e   ', ' n   e   t']

greedy_search
['e e  e  oe', 'e e  e  oe', '          ', '          ', '          ']
tensor([-20.6300, -20.3800,  -9.9200,  -8.0000,  -7.6000], device='cuda:0')

sample
[' pneinhutv', 'Ugnoas h  ', 'feousarrt.', 'ine hn get', '2 cvd ne l']

beam_search
[[' e  e  oe ', ' e        ', '          ', '          ', '          '],
 [' e  e     ', ' e  t     ', '      \n   ', '        . ', '        . '],
 [' e  e  e  ', ' e      t ', '         .', '         .', '         .'],
 [' e  e  t  ', ' e       t', '        \n ', '         K', '         K'],
 [' e  e  e t', ' e       o', '         \n', ' t        ', '    \n     ']]
tensor([[-19.1000, -15.2700,  -9.9200,  -8.0000,  -7.6000],
        [-19.2500, -17.8000, -13.1000, -10.7900, -10.0700],
        [-19.3500, -17.9600, -13.1100, -10.8500, -10.1800],
        [-19.6200


Epoch:    2, Loss: 1.9709
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']

forward
[' e anne  e', 'nhaone  e ', ' tone  e  ', ' ane  e   ', ' ne  e   t']

greedy_search
[' the the t', 'n the the ', ' the the t', ' the the t', ' the the t']
tensor([-11.2300, -10.8000, -10.8700, -10.7900, -11.0800], device='cuda:0')

sample
['sitopascic', 'v\ngif bast', ' opeserera', ' is opann ', 'ask omlet ']

beam_search
[['s the the ', 's the the ', 'e the the ', ' the the t', 'the the th'],
 ['n the the ', 'n the the ', ' the the t', 'the the th', ' the the t'],
 [' the the t', 'e the the ', 'the the th', ' the the p', ' the the c'],
 ['the the th', 'the the th', ' the the p', ' the the a', ' the the p'],
 [' the the p', 'the the to', ' the the a', ' the the c', ' the the a']]
tensor([[-10.6800, -10.5500, -10.6200, -10.7900, -10.8300],
        [-11.1500, -10.8000, -10.8700, -11.0100, -11.0800],
        [-11.2300, -11.0200, -11.4300, -11.7300, -11.9600],
        [-11.8800, -


Epoch:    3, Loss: 1.6086
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']

forward
[' e oothrte', 'n oothrtes', ' tothrtes ', 'toshrtese ', 'tn r es  a']

greedy_search
[' the pasti', 'n the pack', ' the pack ', 'the pack o', 'the pack o']
tensor([-14.7500, -13.5800, -12.5600, -13.3500, -13.5700], device='cuda:0')

sample
['gber ir in', 'icithoct h', '-Q2.\nThoul', 'theraorer ', 'ere, asitr']

beam_search
[['s the the ', 's the the ', ' the the c', 'of the the', 'the the th'],
 ['s the ther', 's of the p', ' the the p', 'the the th', 'the the pa'],
 ['s of the p', 's the ther', ' the the t', 'the the pa', 'the the co'],
 ['s of the c', 's of the c', ' the there', 'the the co', 'the the po'],
 ['s the thes', 's of the t', ' the ther ', 'the the po', 'the the mo']]
tensor([[-10.7900, -10.5000, -10.9800, -10.4300, -10.6900],
        [-11.4500, -11.5300, -10.9900, -10.5000, -11.3600],
        [-11.4600, -11.5700, -11.1700, -11.2600, -11.5300],
        [-11.6200, -


Epoch:    4, Loss: 1.4363
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']

forward
['ne outerte', 'n outhrten', ' tuthrten ', 'tuter ent ', 'on rten  t']

greedy_search
['nt of the ', 'n the pawn', ' the pawn ', 'the pawn t', 'of the paw']
tensor([ -9.4300, -12.9500, -12.1000, -12.2200, -12.0400], device='cuda:0')

sample
['k it by am', 'e ho vat p', ' preant th', 'thaP\nPnarm', 'P-Kt1) In ']

beam_search
[['nt of the ', 't of the p', ' in the th', 'of the the', 'of the the'],
 ['s of the p', 'n of the p', ' in the pa', 'of the Bla', 'of the Bla'],
 ['s in the p', 't of the K', ' in there ', 'the the pa', 'of the pra'],
 ['s of the t', 't of the t', ' in the co', 'of the thi', 'of the Kt-'],
 ['s of the B', 't of the B', ' in the pr', 'the the po', 'of the thi']]
tensor([[ -9.4300, -10.8100, -10.2100,  -9.2600,  -9.5300],
        [-10.5300, -10.9400, -10.9300,  -9.6500,  -9.7800],
        [-10.8700, -11.0800, -11.0200, -11.0500, -11.2500],
        [-10.9500, -


Epoch:    5, Loss: 1.3381
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']

forward
['nt iutente', 'n iutenten', ' iutertent', 'tutertenti', 'orerteneii']

greedy_search
['nt of the ', 'n of the p', ' in the pa', 'the pawns ', 'ore theren']
tensor([ -9.7800, -10.5200, -10.1200, -10.5200, -11.3600], device='cuda:0')

sample
["ce's\n20\nAp", 'k, morjeni', '-3.\n\nIt ad', 'of thawsto', 'it EN ASON']

beam_search
[['nt of the ', 'k in there', ' in there ', 'of the Bla', 'is in the '],
 ['d in there', 'n of the p', ' of the pa', 'for the pa', 'is in ther'],
 ['s in there', 't in the p', ' in the pa', 'for the pr', 'is in the\n'],
 ['nt of ther', 'k of the p', ' in theren', 'for the pl', 'is therent'],
 ['s in the p', 'n of the B', ' in the pr', 'for the co', 'is therere']]
tensor([[ -9.7800, -10.2200,  -9.6300,  -9.6700,  -9.0100],
        [-10.5300, -10.5200, -10.0300, -10.0900, -10.2400],
        [-11.0000, -10.6300, -10.1200, -10.4900, -10.6300],
        [-11.290

In [9]:
importlib.reload(architectures)    
net = architectures.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100,
                                            encoder_layers = 2,
                                            decoder_hidden_units = 100,
                                            decoder_layers = 2)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
test = net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)

test[1][:20]

Net parameters: 408,795


tensor([[-18.8141, -19.7653, -19.8694, -19.9193, -20.2288],
        [-17.4957, -18.6100, -18.6299, -18.9028, -19.8872],
        [-18.3789, -20.2667, -20.3989, -20.7093, -20.7725],
        [-15.6710, -16.2029, -16.3612, -16.9529, -17.5500],
        [-17.8357, -18.7938, -19.3254, -19.7596, -19.8829],
        [-21.1543, -21.5276, -21.6064, -21.8728, -21.8767],
        [-15.6704, -17.1371, -18.1419, -18.5228, -18.5794],
        [-18.4857, -19.1718, -19.3081, -20.3524, -20.7493],
        [-15.7996, -16.2512, -16.4037, -17.7195, -17.9228],
        [-16.0028, -16.6383, -16.8499, -18.0045, -18.2924],
        [-17.7652, -18.0934, -19.1811, -19.1906, -19.2634],
        [-14.9344, -16.1676, -16.7993, -17.7656, -18.0726],
        [-18.5126, -19.2358, -19.5149, -20.0582, -20.1607],
        [-15.9511, -16.5176, -17.1594, -17.1826, -17.2279],
        [-15.3750, -16.4801, -16.6909, -17.0651, -17.0956],
        [-10.9919, -11.4634, -12.0848, -12.1962, -12.2681],
        [-19.6064, -19.7052, -20.0788, -